OSRM:
Pulling data from OSRM:



all shell commands for downloading the osrm docker and running a osrm on our back end:

sh: 

docker pull ghcr.io/project-osrm/osrm-backend:v5.27.1

mkdir -p ~/osrm-data

Then copy the file to this dir, via windows explorer.

sh:

Extract the data:
docker run -t -v ~/osrm-data:/data osrm/osrm-backend osrm-extract -p /opt/car.lua /data/greater-london-latest.osm.pbf

Partition the data:
docker run -t -v ~/osrm-data:/data osrm/osrm-backend osrm-partition /data/greater-london-latest.osrm

Customize the data:
docker run -t -v ~/osrm-data:/data osrm/osrm-backend osrm-customize /data/greater-london-latest.osrm



Then run the backend server:
docker run -d -t -i -p 5000:5000 -v ~/osrm-data:/data osrm/osrm-backend osrm-routed --algorithm mld /data/greater-london-latest.osrm

verify server status:
curl http://localhost:5000/



### The above is for running it first time, version v5.27.1

The following is for shutting down and running again:

* List running containers

docker ps

* Stop the OSRM container
docker stop <container_id>

* (Optional) Remove the OSRM container
docker rm <container_id>

* Run the OSRM backend server
docker run -d -t -i -p 5000:5000 -v ~/osrm-data:/data osrm/osrm-backend osrm-routed --algorithm mld /data/greater-london-latest.osrm


Now the message shows that we have the backend of OSRM running successfully!

time to get the list of lat lon pairs, and query them to the running osrm:

In [1]:
import pandas as pd
import requests
import csv
import time

In [2]:
msoa_london = pd.read_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa/processed/gld_msoa_centroids.csv', encoding='latin1')
# see if the column msoa21cd is unique:
msoa_london['msoa21cd'].nunique() == msoa_london.shape[0]

True

In [3]:
# new dict origin: 1002 maoas
origin_coord_dict = {}
origin_coord = []

# new dict destination:
dest_coord = []
dest_coord_dict = {}

# for reference dict, read the msoa_london, use msoa21cd as key, lat and lon as value:
for i in range(msoa_london.shape[0]):
    msoaID = msoa_london.iloc[i, 3]
    lat = msoa_london.iloc[i, 8]
    lon = msoa_london.iloc[i, 9]
    #origin_coord.append({"lat": lat, "lng": lon})
    origin_coord_dict[msoaID] = {"lat": lat, "lng": lon}
    dest_coord_dict[msoaID] = {"lat": lat, "lng": lon}

In [9]:
origin_coord_dict

{'E02000318': {'lat': 51.48826506306727, 'lng': 0.0956181263047325},
 'E02000388': {'lat': 51.48315729022629, 'lng': -0.1984452219606495},
 'E02000652': {'lat': 51.41504113246437, 'lng': -0.1380462958677474},
 'E02000963': {'lat': 51.52907905830605, 'lng': -0.209370884304258},
 'E02000651': {'lat': 51.4191529314827, 'lng': -0.1259419896669564},
 'E02000216': {'lat': 51.36949206068002, 'lng': -0.077636700171988},
 'E02000746': {'lat': 51.51620181303971, 'lng': 0.0774631221982},
 'E02000103': {'lat': 51.56360242260222, 'lng': -0.2344250853261522},
 'E02000828': {'lat': 51.474611437273566, 'lng': -0.0673270348400329},
 'E02000628': {'lat': 51.466601391774674, 'lng': -0.1124073736023934},
 'E02000380': {'lat': 51.4984182398741, 'lng': -0.2305090479379697},
 'E02000571': {'lat': 51.53792786833988, 'lng': -0.0936864361978526},
 'E02000590': {'lat': 51.49332975613745, 'lng': -0.1691014538105725},
 'E02000432': {'lat': 51.572461188240176, 'lng': -0.10859264639662},
 'E02000615': {'lat': 51.370

In [4]:
# transform the dict to dataframe
origin_coord_df = pd.DataFrame(origin_coord_dict).T

In [5]:
origin_coord_df.head()

,lat,lng
E02000318,51.488265,0.095618
E02000388,51.483157,-0.198445
E02000652,51.415041,-0.138046
E02000963,51.529079,-0.209371
E02000651,51.419153,-0.125942


In [26]:
# save origin_coord_df to csv:
origin_coord_df.to_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa//origin_dest_coord_id_lookup.csv')

In [6]:
# rename lng to lon:
origin_coord_df.rename(columns={"lng": "lon"}, inplace=True)

In [7]:
# name first column as msoaID:
origin_coord_df.reset_index(inplace=True) 
origin_coord_df.rename(columns={"index": "msoaID"}, inplace=True)

In [8]:
# Generate all possible OD pairs
od_pairs = pd.merge(origin_coord_df, origin_coord_df, how='cross')
od_pairs.columns = ['origin_msoaID', 'origin_lat', 'origin_lon', 'destination_msoaID', 'destination_lat', 'destination_lon']

In [28]:
od_pairs.to_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa/all_possible_od_pairs.csv')

In [9]:
od_pairs.head()

,origin_msoaID,origin_lat,origin_lon,destination_msoaID,destination_lat,destination_lon
0,E02000318,51.488265,0.095618,E02000318,51.488265,0.095618
1,E02000318,51.488265,0.095618,E02000388,51.483157,-0.198445
2,E02000318,51.488265,0.095618,E02000652,51.415041,-0.138046
3,E02000318,51.488265,0.095618,E02000963,51.529079,-0.209371
4,E02000318,51.488265,0.095618,E02000651,51.419153,-0.125942


In [10]:
# how many od pairs are there?
od_pairs.shape[0]

1004004

now code the query functions and get ready to send the resquest:

In [11]:
# Function to get travel time between two coordinates
def get_travel_time(lat1, lon1, lat2, lon2):
    url = f'http://localhost:5000/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['code'] == 'Ok':
            return data['routes'][0]['duration']
    return None

In [12]:
results = []
# print start time:
print(time.strftime("%H:%M:%S", time.localtime()))

for index, row in od_pairs.iterrows():
    origin_lat = row['origin_lat']
    origin_lon = row['origin_lon']
    destination_lat = row['destination_lat']
    destination_lon = row['destination_lon']
    
    travel_time = get_travel_time(origin_lat, origin_lon, destination_lat, destination_lon)
    
    results.append({
        'origin_msoaID': row['origin_msoaID'],
        'destination_msoaID': row['destination_msoaID'],
        'origin_lat': origin_lat,
        'origin_lon': origin_lon,
        'destination_lat': destination_lat,
        'destination_lon': destination_lon,
        'travel_time': travel_time
    })

    if index % 100 == 0:
        print(f'Processed {index} pairs')

    time.sleep(0.1)

# print end time:
print(time.strftime("%H:%M:%S", time.localtime()))

21:34:09
Processed 0 pairs
Processed 100 pairs
Processed 200 pairs
Processed 300 pairs
Processed 400 pairs
Processed 500 pairs
Processed 600 pairs
Processed 700 pairs
Processed 800 pairs
Processed 900 pairs
Processed 1000 pairs
Processed 1100 pairs
Processed 1200 pairs
Processed 1300 pairs
Processed 1400 pairs
Processed 1500 pairs
Processed 1600 pairs
Processed 1700 pairs
Processed 1800 pairs
Processed 1900 pairs
Processed 2000 pairs
Processed 2100 pairs
Processed 2200 pairs
Processed 2300 pairs
Processed 2400 pairs
Processed 2500 pairs
Processed 2600 pairs
Processed 2700 pairs
Processed 2800 pairs
Processed 2900 pairs
Processed 3000 pairs
Processed 3100 pairs
Processed 3200 pairs
Processed 3300 pairs
Processed 3400 pairs
Processed 3500 pairs
Processed 3600 pairs
Processed 3700 pairs
Processed 3800 pairs
Processed 3900 pairs
Processed 4000 pairs
Processed 4100 pairs
Processed 4200 pairs
Processed 4300 pairs
Processed 4400 pairs
Processed 4500 pairs
Processed 4600 pairs
Processed 4700 p

Total running time: 1 day,5 hours,48 minutes, and 47 seconds
i.e. 
29 hours 48 minutes, and 47 seconds

In [13]:
results_df = pd.DataFrame(results)
results_df.to_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa/osrm_travel_times.csv', index=False)


In [ ]:
# Prepare a list to store results
results = []

# Iterate over each row in the DataFrame and get travel times
for index, row in origin_coord_df.iterrows():
    origin_lat = row['lat']
    origin_lon = row['lon']
    destination_lat = row['lat']
    destination_lon = row['lon']
    
    travel_time = get_travel_time(origin_lat, origin_lon, destination_lat, destination_lon)
    
    # Append the result as a dictionary
    results.append({
        'origin_lat': origin_lat,
        'origin_lon': origin_lon,
        'destination_lat': destination_lat,
        'destination_lon': destination_lon,
        'travel_time': travel_time
    })

    # Print progress
    if index % 100 == 0:
        print(f'Processed {index} pairs')

    # To prevent overwhelming the server, you might want to add a small delay
    time.sleep(0.1)  # 100 ms delay between requests

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to a CSV file
results_df.to_csv('osrm_travel_times.csv', index=False)

print('Completed processing all OD pairs.')


In [ ]:


# Function to get travel time between two coordinates
def get_travel_time(lat1, lon1, lat2, lon2):
    url = f'http://localhost:5000/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false'
    response = requests.get(url)
    data = response.json()
    if data['code'] == 'Ok':
        return data['routes'][0]['duration']
    else:
        return None

# Read the origin-destination pairs from CSV
with open('od_pairs.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        origin_lat = row['origin_lat']
        origin_lon = row['origin_lon']
        destination_lat = row['destination_lat']
        destination_lon = row['destination_lon']
        
        travel_time = get_travel_time(origin_lat, origin_lon, destination_lat, destination_lon)
        print(f'Travel time from ({origin_lat}, {origin_lon}) to ({destination_lat}, {destination_lon}) is {travel_time} seconds')


In [2]:
import osrm
import pandas as pd

In [3]:
msoa_london = pd.read_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa/processed/gld_msoa_centroids.csv', encoding='latin1')
# see if the column msoa21cd is unique:
msoa_london['msoa21cd'].nunique() == msoa_london.shape[0]

True

In [4]:
list_coord = []

list_id= []

# for reference dict, read the msoa_london, use msoa21cd as key, lat and lon as value:
for i in range(msoa_london.shape[0]):
    msoaID = msoa_london.iloc[i, 3]
    lat = msoa_london.iloc[i, 8]
    lon = msoa_london.iloc[i, 9]
    list_id.append(msoaID)
    list_coord.append((lat, lon))
    

In [6]:

%pip install osrm
time_matrix, snapped_coords = osrm.table(list_coord, ids_origin=list_id,
                                  				  output='dataframe')


ImportError: cannot import name 'table' from 'osrm' (/home/mengyuwsl/.local/lib/python3.10/site-packages/osrm.py)

In [ ]:
import requests

def get_osrm_matrix(coordinates, annotations="duration"):
    # Join the coordinates into the format required by OSRM
    coordinates_str = ';'.join([f"{lon},{lat}" for lon, lat in coordinates])
    
    # Construct the request URL
    url = f"http://router.project-osrm.org/table/v1/driving/{coordinates_str}?annotations={annotations}"
    
    # Send the request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

# List of coordinates (longitude, latitude)
coordinates = [
    (13.388860, 52.517037),
    (13.397634, 52.529407),
    (13.428555, 52.523219)
]

# Get the duration matrix
result = get_osrm_matrix(coordinates, annotations="duration")

if result:
    print("Duration Matrix:")
    for row in result['durations']:
        print(row)

# Get the distance matrix
result = get_osrm_matrix(coordinates, annotations="distance")

if result:
    print("\nDistance Matrix:")
    for row in result['distances']:
        print(row)
